In [57]:
# === Core scientific stack ===
import sys, os, re, json, time, random, warnings, logging
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import logging

# === Scikit-learn (keep only what’s needed for time-series and metrics) ===
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_percentage_error as mape

# === Optuna for hyperparameter optimization ===
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

# === Persistence & utilities ===
import joblib
from tqdm import tqdm  # progress bar for multi-sensor training

# Global configuration
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Use half of available CPU cores for XGBoost
N_JOBS = os.cpu_count()

# Output directories for saving models, figures, and metrics
ARTIFACTS_DIR = Path("artifacts")
MODELS_DIR = Path("models")
FIGS_DIR = Path("figures")
for d in [ARTIFACTS_DIR, MODELS_DIR, FIGS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Basic logging setup — helpful when training multiple sensors
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%H:%M:%S",
)

optuna.logging.set_verbosity(optuna.logging.WARNING)   # 只保留 warning 以上
optuna.logging.disable_default_handler()               # 关掉 Optuna 自己的 stdout handler
xgb.set_config(verbosity=0)  

# Suppress non-critical warnings
warnings.filterwarnings("ignore")


In [58]:
# Version check
import sklearn
print("Python:", sys.version.split()[0])
print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)
print("scikit-learn:", sklearn.__version__)
print("XGBoost:", xgb.__version__)
print("Optuna:", optuna.__version__)
print("CPU cores:", os.cpu_count(), "| N_JOBS:", N_JOBS)

Python: 3.12.9
NumPy: 2.3.3
Pandas: 2.3.2
scikit-learn: 1.7.2
XGBoost: 3.0.1
Optuna: 4.5.0
CPU cores: 22 | N_JOBS: 22


In [59]:
# ============================================================
# 全局设置（合并版）：路径 / 配置 / 数据读取 / 列自动识别
# 说明：
# - 使用一个 CFG 字典集中管理调参与切分开关
# - 统一创建输出目录
# - 读取CSV并设置时间索引
# - 自动识别：传感器列 / 时间派生特征 / 外生变量及其 isna 指示
# - 去掉重复变量命名（仅保留 TIME_COLS / EXOG_ALL / SENSOR_COLS）
# ============================================================

# ---------- 1) 输入文件 ----------
# 注：如需相对路径可改为 Path("training_dataset_tomtom_vessel_ready.csv")
DATA_PATH = Path("C:/Users/elvinli/OneDrive/CodeProjects/TIL6022-Group23-SAIL-Dashboard/outputs_fast/training_dataset_tomtom_vessel_ready.csv")

# ---------- 2) 输出目录 ----------
ARTIFACTS_DIR = Path("artifacts")
MODELS_DIR = Path("models")
FIGS_DIR = Path("figures")
for d in [ARTIFACTS_DIR, MODELS_DIR, FIGS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ---------- 3) 统一配置 ----------
# 依赖：SEED / N_JOBS 已在导入单元中定义
CFG = {
    # 随机性与并行
    "seed": SEED,
    "n_jobs": N_JOBS,

    # 数据切分
    "test_ratio": 0.20,          # 测试集比例（按时间末尾切分）
    "target_shift": 0,           # 若要预测 t+1，可设为 1；当前保持 0 预测当期

    # Optuna 调参
    "use_optuna": True,          # 是否为每个传感器开启调参
    "optuna_n_trials": 50,       # 每个传感器的 trial 数（先轻量，后续可加大）
    "trial_timeout_sec": 120,    # 每个 trial 的最长时间，避免卡死
    "early_stopping_rounds": 100,# XGB 的 early stopping

    # 滞后与防泄漏
    "default_max_lag": 6,        # 若不搜索 max_lag，用该默认值
    "gap_equals_max_lag": True,  # 验证前的 gap = max_lag，严格防泄漏
}

# ---------- 4) XGBoost 默认参数（Optuna会在此基础上搜索/覆盖） ----------
XGB_BASE_PARAMS = {
    "objective": "reg:squarederror",
    "tree_method": "hist",
    "n_estimators": 500,
    "learning_rate": 0.05,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_lambda": 1.0,
    "reg_alpha": 0.0,
    "random_state": CFG["seed"],
    "n_jobs": CFG["n_jobs"],
}

# ---------- 5) 读取数据并设置时间索引 ----------
df = pd.read_csv(DATA_PATH)
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.sort_values("timestamp").set_index("timestamp")

# ---------- 6) 自动识别列（仅选择存在的列，避免 KeyError） ----------
all_cols = df.columns.tolist()

# 6.1 传感器列（按你的命名约定：以 "__sensor" 结尾）
SENSOR_COLS = [c for c in all_cols if c.endswith("__sensor")]

# 6.2 时间派生特征（只保留在数据表中确实存在的）
_CANDIDATE_TIME = ["hour", "minute", "weekday", "is_weekend", "week", "minute_of_day"]
TIME_COLS = [c for c in _CANDIDATE_TIME if c in df.columns]

# 6.3 外生变量及其 isna 指示（只保留存在的）
_CANDIDATE_EXOG = ["traffic_level_mean_tomtom", "vessel_count_vessel", "vessel_avg_speed_vessel"]
EXOG_COLS = [c for c in _CANDIDATE_EXOG if c in df.columns]
EXOG_ISNA_COLS = [f"{c}_isna" for c in _CANDIDATE_EXOG if f"{c}_isna" in df.columns]
EXOG_ALL = EXOG_COLS + EXOG_ISNA_COLS  # 一并送入模型，让树学习“缺失”信息

# 6.4 要训练的传感器清单（可先抽样一部分进行干跑）
# TARGET_SENSORS = SENSOR_COLS[:5]   # 只跑前5个做冒烟测试
TARGET_SENSORS = SENSOR_COLS         # 生产模式：全体传感器

# ---------- 7) 评估与持久化文件 ----------
METRICS_FILE = ARTIFACTS_DIR / "metrics_summary.csv"  # 训练评估总表
OPTUNA_DB = ARTIFACTS_DIR / "optuna_study.db"         # Optuna RDB（可选）

# ---------- 8) 快速日志 ----------
logging.info(
    f"Rows: {len(df)} | Sensors: {len(SENSOR_COLS)} | "
    f"Time features: {len(TIME_COLS)} | Exogenous: {len(EXOG_ALL)}"
)
logging.info(f"Sample sensors: {SENSOR_COLS[:5]}")


2025-11-05 16:41:27,639 | INFO | Rows: 2400 | Sensors: 37 | Time features: 6 | Exogenous: 6
2025-11-05 16:41:27,639 | INFO | Sample sensors: ['CMSA-GAKH-01__sensor', 'CMSA-GAWW-11__sensor', 'CMSA-GAWW-12__sensor', 'CMSA-GAWW-13__sensor', 'CMSA-GAWW-14__sensor']


In [60]:
def build_dataset_for_sensor(
    df,
    sensor_col: str,
    max_lag: int,
    exog_cols: list,
    time_cols: list,
    target_shift: int = 0,
    add_rolling: bool = False,
    rolling_windows: tuple = (3, 6, 12),
    add_diff: bool = False,
    exog_lags: int = 0,
):
    """
    Build a time-series training dataset for one sensor.

    Parameters
    ----------
    df : pd.DataFrame
        The full dataframe. Index must be a DatetimeIndex (timestamp).
    sensor_col : str
        The name of the target sensor column (e.g. 'CMSA-GAKH-01__sensor').
    max_lag : int
        The maximum number of lag features to create (1..max_lag).
    exog_cols : list[str]
        List of exogenous variables and their _isna flags (will auto-filter only those present in df).
    time_cols : list[str]
        List of time-derived features (only those present in df will be used).
    target_shift : int, default=0
        How many steps ahead to predict (0 = current step, 1 = next timestamp, etc.).
    add_rolling : bool, default=False
        Whether to add rolling mean/std features for the target series.
    rolling_windows : tuple[int], default=(3, 6, 12)
        Window sizes for rolling features (each unit equals one sample interval).
    add_diff : bool, default=False
        Whether to include first-order differencing of the target.
    exog_lags : int, default=0
        Whether to include lagged versions of exogenous variables (0 = none).

    Returns
    -------
    X : pd.DataFrame
        The feature matrix, fully aligned and cleaned.
    y : pd.Series
        The target vector, aligned with X.
    feature_names : list[str]
        List of feature names used in the model.
    meta : dict
        Metadata about the dataset (sensor, n_samples, max_lag, etc.).
    """

    # --- 0) Basic validation ---
    if sensor_col not in df.columns:
        raise KeyError(f"Target column {sensor_col} not found in dataframe.")
    if max_lag < 1:
        raise ValueError("max_lag must be >= 1")
    if not isinstance(df.index, pd.DatetimeIndex):
        raise TypeError("df.index must be a DatetimeIndex (set timestamp as index first).")

    # --- 1) Copy only relevant columns to avoid modifying original df ---
    keep_cols = [sensor_col]
    time_cols = [c for c in time_cols if c in df.columns]
    exog_cols = [c for c in exog_cols if c in df.columns]
    keep_cols += time_cols + exog_cols
    work = df[keep_cols].copy()

    # --- 2) Create lag features for the target variable ---
    lag_cols = []
    for k in range(1, max_lag + 1):
        colk = f"{sensor_col}_lag{k}"
        work[colk] = work[sensor_col].shift(k)
        lag_cols.append(colk)

    # --- 3) Add first-order difference (optional) ---
    diff_cols = []
    if add_diff:
        cold = f"{sensor_col}_diff1"
        work[cold] = work[sensor_col].diff(1)
        diff_cols.append(cold)

    # --- 4) Add rolling mean/std features (optional) ---
    roll_cols = []
    if add_rolling:
        for w in rolling_windows:
            cm = f"{sensor_col}_rollmean{w}"
            cs = f"{sensor_col}_rollstd{w}"
            work[cm] = work[sensor_col].rolling(window=w, min_periods=w).mean()
            work[cs] = work[sensor_col].rolling(window=w, min_periods=w).std()
            roll_cols += [cm, cs]

    # --- 5) Add lagged versions of exogenous variables (optional) ---
    exog_lag_cols = []
    if exog_lags and exog_lags > 0:
        for c in exog_cols:
            for k in range(1, exog_lags + 1):
                colk = f"{c}_lag{k}"
                work[colk] = work[c].shift(k)
                exog_lag_cols.append(colk)

    # --- 6) Create target vector y ---
    # Shift the target forward if predicting future values (target_shift > 0)
    y = work[sensor_col].shift(-target_shift) if target_shift > 0 else work[sensor_col]

    # --- 7) Define final feature order ---
    feature_cols = []
    feature_cols += time_cols          # time-based features
    feature_cols += exog_cols          # exogenous variables + isna flags
    feature_cols += exog_lag_cols      # lagged exogenous (if any)
    feature_cols += lag_cols           # lagged target values
    feature_cols += diff_cols          # difference features
    feature_cols += roll_cols          # rolling features

    X = work[feature_cols]

    # --- 8) Align and clean missing values ---
    # Lagging and rolling produce NaNs at the start; shifting target adds NaNs at the end.
    # Drop any rows with missing data to keep X and y aligned.
    data = pd.concat([X, y.rename("y")], axis=1)
    data = data.dropna(axis=0, how="any")

    # Align X and y
    y = data["y"]
    X = data.drop(columns=["y"])

    # --- 9) Metadata for traceability ---
    meta = {
        "sensor": sensor_col,
        "max_lag": max_lag,
        "target_shift": target_shift,
        "n_samples": len(X),
        "n_features": X.shape[1],
        "used_time_cols": time_cols,
        "used_exog_cols": exog_cols,
        "added_exog_lags": exog_lags,
        "added_diff": add_diff,
        "added_rolling": add_rolling,
        "rolling_windows": rolling_windows if add_rolling else (),
    }

    return X, y, feature_cols, meta


In [61]:
def rolling_time_series_splits(
    X,
    y,
    n_splits: int = 3,
    test_size: float = 0.2,
    gap: int = 0,
    mode: str = "expanding",
    min_train_ratio: float = 0.4,
):
    """
    Generate multiple time-based splits (rolling or expanding window)
    for time-series cross-validation with a gap before each validation set.

    Parameters
    ----------
    X : pd.DataFrame
        Feature matrix, must be time-ordered (DatetimeIndex or sorted index).
    y : pd.Series
        Target vector, aligned with X (same index).
    n_splits : int, default=3
        Number of folds (commonly 3–5).
    test_size : float, default=0.2
        Fraction of total samples to allocate for each validation set.
    gap : int, default=0
        Number of samples to skip between train and validation to prevent leakage.
    mode : {"expanding", "sliding"}, default="expanding"
        - "expanding": training set starts at the beginning and grows each fold.
        - "sliding" : fixed-length training window sliding forward.
    min_train_ratio : float, default=0.4
        Minimum proportion of samples for the first training window.

    Yields
    ------
    (X_train, y_train, X_val, y_val, fold_info)
        - X_train, y_train : training subset
        - X_val, y_val     : validation subset
        - fold_info : dict with metadata (fold index, sizes, date range)
    """
    import math

    # --- Basic validation ---
    if len(X) != len(y):
        raise ValueError("X and y must have the same length and aligned index.")
    n = len(X)
    if n < 10:
        raise ValueError("Too few samples for time-series splitting.")
    if not 0 < test_size < 1:
        raise ValueError("test_size must be in (0, 1).")

    # --- Define sizes ---
    min_train = max(1, int(n * min_train_ratio))
    val_len = max(1, int(n * test_size))

    for fold in range(1, n_splits + 1):
        if mode == "expanding":
            # Expanding window: train grows each fold
            train_end = min_train + (fold - 1) * max(
                1, (n - min_train - val_len) // max(1, n_splits)
            )
            val_start = train_end + gap
            val_end = val_start + val_len

        elif mode == "sliding":
            # Sliding window: train window moves forward, fixed length
            train_len = max(min_train, n - (n_splits - fold + 1) * (val_len + gap))
            train_start = max(
                0, n - (train_len + (n_splits - fold + 1) * (val_len + gap))
            )
            train_end = train_start + train_len
            val_start = train_end + gap
            val_end = val_start + val_len

        else:
            raise ValueError("mode must be 'expanding' or 'sliding'.")

        # Skip invalid ranges
        if val_end > n or val_start >= n or train_end <= 0:
            continue

        # Slice the data (no shuffling)
        X_tr = X.iloc[:train_end] if mode == "expanding" else X.iloc[train_start:train_end]
        y_tr = y.iloc[:train_end] if mode == "expanding" else y.iloc[train_start:train_end]
        X_val = X.iloc[val_start:val_end]
        y_val = y.iloc[val_start:val_end]

        # Skip empty splits
        if len(X_val) == 0 or len(X_tr) == 0:
            continue

        # Metadata for logging
        fold_info = {
            "fold": fold,
            "mode": mode,
            "gap": gap,
            "n_train": len(X_tr),
            "n_val": len(X_val),
            "train_end_idx": train_end,
            "val_start_idx": val_start,
            "val_end_idx": val_end,
            "train_index_start": X_tr.index[0],
            "train_index_end": X_tr.index[-1],
            "val_index_start": X_val.index[0],
            "val_index_end": X_val.index[-1],
        }

        yield X_tr, y_tr, X_val, y_val, fold_info


In [62]:
def train_one_sensor(
    sensor_col: str,
    df,
    cfg,
    exog_cols,
    time_cols,
    xgb_base_params,
    study_db=None,
):
    """
    Train and evaluate one sensor model with multi-fold time-series CV and Optuna tuning.

    Parameters
    ----------
    sensor_col : str
        The sensor column name (e.g., 'CMSA-GAKH-01__sensor').
    df : pd.DataFrame
        Full dataset (must include all exogenous/time features and the target sensor column).
    cfg : dict
        Global configuration dictionary (contains optuna_n_trials, test_ratio, default_max_lag, etc.).
    exog_cols : list[str]
        List of exogenous variable names and _isna flags.
    time_cols : list[str]
        List of time-based feature names.
    xgb_base_params : dict
        Base XGBoost parameters (Optuna will search around them).
    study_db : Path or None, optional
        Path to an Optuna SQLite study DB (if None, a temporary in-memory study will be used).

    Returns
    -------
    metrics : dict
        Aggregated metrics across folds (mean R2, RMSE, MAE, best_params, etc.).
    """

    import numpy as np
    import xgboost as xgb
    from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

    # --------------------------------------------------
    # 1) Build dataset
    # --------------------------------------------------
    X, y, feat_names, meta = build_dataset_for_sensor(
        df=df,
        sensor_col=sensor_col,
        max_lag=cfg["default_max_lag"],
        exog_cols=exog_cols,
        time_cols=time_cols,
        target_shift=cfg["target_shift"],
    )

    # --------------------------------------------------
    # 2) Define Optuna objective
    # --------------------------------------------------
    def objective(trial):
        # Search space around base parameters
        params = xgb_base_params.copy()
        params.update(
            {
                "max_depth": trial.suggest_int("max_depth", 3, 10),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                "subsample": trial.suggest_float("subsample", 0.6, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
                "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
                "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10.0, log=True),
                "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
                "max_lag": trial.suggest_int("max_lag", 3, 12),
            }
        )

        # Rebuild dataset if lag length changes
        X_trial, y_trial, _, _ = build_dataset_for_sensor(
            df=df,
            sensor_col=sensor_col,
            max_lag=params["max_lag"],
            exog_cols=exog_cols,
            time_cols=time_cols,
            target_shift=cfg["target_shift"],
        )

        # Multi-fold CV evaluation
        scores_r2, scores_rmse, scores_mae = [], [], []
        gap = params["max_lag"] if cfg.get("gap_equals_max_lag", True) else 0

        for X_tr, y_tr, X_val, y_val, fi in rolling_time_series_splits(
            X_trial,
            y_trial,
            n_splits=3,
            test_size=cfg["test_ratio"],
            gap=gap,
            mode="expanding",
            min_train_ratio=0.4,
        ):
            params_with_metric = params.copy()
            params_with_metric["eval_metric"] = "rmse"
            
            allowed_keys = {
                "n_estimators", "max_depth", "learning_rate",
                "subsample", "colsample_bytree",
                "reg_lambda", "reg_alpha", "min_child_weight",
                "gamma", "max_bin", "tree_method",
                "n_jobs", "random_state", "eval_metric"
            }
            model_params = {k: v for k, v in params_with_metric.items() if k in allowed_keys}

            model = xgb.XGBRegressor(**model_params)

            model.fit(
                X_tr,
                y_tr,
                eval_set=[(X_val, y_val)],
                verbose=False,
            )  

            preds = model.predict(X_val)
            scores_r2.append(r2_score(y_val, preds))
            scores_rmse.append(np.sqrt(mean_squared_error(y_val, preds)))
            scores_mae.append(mean_absolute_error(y_val, preds))

        # Average fold results (optimize by RMSE)
        mean_rmse = np.mean(scores_rmse)
        trial.set_user_attr("r2", np.mean(scores_r2))
        trial.set_user_attr("mae", np.mean(scores_mae))
        return mean_rmse

    # --------------------------------------------------
    # 3) Run Optuna optimization
    # --------------------------------------------------
    study_name = f"xgb_{sensor_col}"
    storage = f"sqlite:///{study_db}" if study_db is not None else None
    study = optuna.create_study(
        study_name=study_name,
        direction="minimize",
        sampler=optuna.samplers.TPESampler(seed=cfg["seed"]),
        pruner=optuna.pruners.MedianPruner(),
        storage=storage,
        load_if_exists=True,
    )
    study.optimize(objective, n_trials=cfg["optuna_n_trials"], timeout=cfg["trial_timeout_sec"])

    best_params = study.best_trial.params
    best_rmse = study.best_value
    best_r2 = study.best_trial.user_attrs.get("r2", None)
    best_mae = study.best_trial.user_attrs.get("mae", None)

    # --------------------------------------------------
    # 4) Retrain final model on all data with best params
    # --------------------------------------------------
    final_params = xgb_base_params.copy()
    final_params.update(best_params)
    final_params["eval_metric"] = "rmse"

    allowed_keys = {
        "n_estimators", "max_depth", "learning_rate",
        "subsample", "colsample_bytree",
        "reg_lambda", "reg_alpha", "min_child_weight",
        "gamma", "max_bin", "tree_method",
        "n_jobs", "random_state", "eval_metric"
    }

    final_model_params = {k: v for k, v in final_params.items() if k in allowed_keys}
    final_model = xgb.XGBRegressor(**final_model_params)
    final_model.fit(X, y)

    # --------------------------------------------------
    # 5) Save model and record metrics
    # --------------------------------------------------
    model_path = MODELS_DIR / f"{sensor_col}.json"
    final_model.save_model(model_path)

    metrics = {
        "sensor": sensor_col,
        "r2_mean": best_r2,
        "rmse_mean": best_rmse,
        "mae_mean": best_mae,
        "n_samples": len(X),
        "n_features": X.shape[1],
        "best_params": best_params,
        "model_path": str(model_path),
    }

    return metrics


In [ ]:
# ============================================================
# 批量训练循环（Batch Training Loop）
# ============================================================

# 说明：
# - 依次遍历 TARGET_SENSORS 中的每个传感器
# - 调用 train_one_sensor() 进行多折调参与训练
# - 将每个传感器的结果追加保存到 metrics_summary.csv
# - 若个别传感器训练失败，不影响整体流程
import time

# 初始化结果记录表
metrics_records = []

# 打印总体任务信息
logging.info(f"🚀 Starting batch training for {len(TARGET_SENSORS)} sensors...")

start_all = time.time()

for i, sensor in enumerate(TARGET_SENSORS, start=1):
    logging.info(f"\n==============================")
    logging.info(f"[{i}/{len(TARGET_SENSORS)}] Training sensor: {sensor}")

    start_time = time.time()

    try:
        # 调用单传感器训练封装函数
        metrics = train_one_sensor(
            sensor_col=sensor,
            df=df,
            cfg=CFG,
            exog_cols=EXOG_ALL,
            time_cols=TIME_COLS,
            xgb_base_params=XGB_BASE_PARAMS,
            study_db=OPTUNA_DB,
        )
    
        # 增加时间信息与状态
        metrics["status"] = "success"
        metrics["train_time_sec"] = round(time.time() - start_time, 2)
        metrics_records.append(metrics)

        logging.info(
            f"✅ {sensor} finished | "
            f"R²={metrics.get('r2_mean'):.4f}, RMSE={metrics.get('rmse_mean'):.4f}, "
            f"MAE={metrics.get('mae_mean'):.4f}, Time={metrics['train_time_sec']}s"
        )

    except Exception as e:
        # 训练失败的传感器会被记录下来
        logging.error(f"❌ {sensor} failed: {e}")
        metrics_records.append({
            "sensor": sensor,
            "status": "failed",
            "error": str(e),
            "r2_mean": None,
            "rmse_mean": None,
            "mae_mean": None,
            "train_time_sec": round(time.time() - start_time, 2)
        })

# ------------------------------------------------------------
# 汇总保存结果
# ------------------------------------------------------------
df_metrics = pd.DataFrame(metrics_records)
df_metrics.to_csv(METRICS_FILE, index=False)

total_time = round(time.time() - start_all, 2)
logging.info(f"\n🏁 All sensors completed in {total_time}s")
logging.info(f"Results saved to: {METRICS_FILE}")

# 可选：打印训练成功/失败统计
n_success = (df_metrics["status"] == "success").sum()
n_failed = (df_metrics["status"] == "failed").sum()
logging.info(f"Training summary → Success: {n_success}, Failed: {n_failed}")


2025-11-05 16:41:27,689 | INFO | 🚀 Starting batch training for 37 sensors...
2025-11-05 16:41:27,690 | INFO | 
2025-11-05 16:41:27,691 | INFO | [1/37] Training sensor: CMSA-GAKH-01__sensor
2025-11-05 16:42:38,283 | ERROR | ❌ CMSA-GAKH-01__sensor failed: 'train_time_sec'
2025-11-05 16:42:38,285 | INFO | 
2025-11-05 16:42:38,285 | INFO | [2/37] Training sensor: CMSA-GAWW-11__sensor
2025-11-05 16:43:51,955 | ERROR | ❌ CMSA-GAWW-11__sensor failed: 'train_time_sec'
2025-11-05 16:43:51,956 | INFO | 
2025-11-05 16:43:51,956 | INFO | [3/37] Training sensor: CMSA-GAWW-12__sensor


Trial 5 failed with parameters: {'max_depth': 7, 'learning_rate': 0.11114989443094977, 'subsample': 0.608233797718321, 'colsample_bytree': 0.9879639408647978, 'reg_lambda': 2.1368329072358767, 'reg_alpha': 0.0070689749506246055, 'min_child_weight': 2, 'max_lag': 4} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\elvinli\.conda\envs\TIL6022-25\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\elvinli\AppData\Local\Temp\ipykernel_37504\3550949636.py", line 108, in objective
    model.fit(
  File "c:\Users\elvinli\.conda\envs\TIL6022-25\Lib\site-packages\xgboost\core.py", line 729, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\Users\elvinli\.conda\envs\TIL6022-25\Lib\site-packages\xgboost\sklearn.py", line 1247, in fit
    self._Booster = train(
                    ^^^^^^
  File "c:\Users\elvinli\.cond

KeyboardInterrupt: 

In [ ]:
# ============================================================
# 模型性能可视化脚本 (Visualization of model performance)
# ============================================================

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# --- 1) 读取结果文件 ---
metrics_path = METRICS_FILE
if not metrics_path.exists():
    raise FileNotFoundError(f"Metrics file not found: {metrics_path}")

dfm = pd.read_csv(metrics_path)
dfm_success = dfm[dfm["status"] == "success"].copy()

if dfm_success.empty:
    raise ValueError("No successful models found — please check training results.")

# --- 2) 基本统计信息 ---
print("✅ Loaded metrics summary:", metrics_path)
print(f"Successful models: {len(dfm_success)} / {len(dfm)}")
print(f"Average R²: {dfm_success['r2_mean'].mean():.3f}")
print(f"Average RMSE: {dfm_success['rmse_mean'].mean():.3f}")
print(f"Average MAE: {dfm_success['mae_mean'].mean():.3f}")

# --- 3) 统一样式设置 ---
sns.set(style="whitegrid", font="DejaVu Sans", rc={"axes.titlesize": 14, "axes.labelsize": 12})

# --- 4) R² 排名条形图 ---
plt.figure(figsize=(12, 6))
df_plot = dfm_success.sort_values("r2_mean", ascending=False)
sns.barplot(x="r2_mean", y="sensor", data=df_plot, palette="viridis")
plt.title("Model Performance by Sensor (R² Score)")
plt.xlabel("R² (higher is better)")
plt.ylabel("Sensor ID")
plt.tight_layout()
plt.savefig(FIGS_DIR / "sensor_r2_ranking.png", dpi=300)
plt.show()

# --- 5) RMSE 排名条形图 ---
plt.figure(figsize=(12, 6))
df_plot = dfm_success.sort_values("rmse_mean", ascending=True)
sns.barplot(x="rmse_mean", y="sensor", data=df_plot, palette="rocket_r")
plt.title("Model RMSE by Sensor (lower is better)")
plt.xlabel("RMSE")
plt.ylabel("Sensor ID")
plt.tight_layout()
plt.savefig(FIGS_DIR / "sensor_rmse_ranking.png", dpi=300)
plt.show()

# --- 6) MAE 排名条形图 ---
plt.figure(figsize=(12, 6))
df_plot = dfm_success.sort_values("mae_mean", ascending=True)
sns.barplot(x="mae_mean", y="sensor", data=df_plot, palette="crest")
plt.title("Model MAE by Sensor (lower is better)")
plt.xlabel("MAE")
plt.ylabel("Sensor ID")
plt.tight_layout()
plt.savefig(FIGS_DIR / "sensor_mae_ranking.png", dpi=300)
plt.show()

# --- 7) 指标分布箱线图 ---
plt.figure(figsize=(10, 5))
sns.boxplot(data=dfm_success[["r2_mean", "rmse_mean", "mae_mean"]], palette="Set2")
plt.title("Distribution of Model Performance Metrics")
plt.ylabel("Metric value")
plt.tight_layout()
plt.savefig(FIGS_DIR / "metrics_distribution_boxplot.png", dpi=300)
plt.show()

# --- 8) 输出 Top-N 最佳与最差模型表格 ---
TOP_N = 5
print("\n🏆 Top models by R²:")
display(dfm_success.nlargest(TOP_N, "r2_mean")[["sensor", "r2_mean", "rmse_mean", "mae_mean"]])

print("\n⚠️ Worst models by R²:")
display(dfm_success.nsmallest(TOP_N, "r2_mean")[["sensor", "r2_mean", "rmse_mean", "mae_mean"]])


ValueError: No successful models found — please check training results.